本节将介绍在训练过程中如何更好地访问并控制模型内部过程的方法。使用model.fit()或model.fit_generator()在一个大型数据集上启动数十轮的训练，有点类似于扔一架纸飞机，一开始给它一点推力，之后你便再也无法控制其飞行轨迹或着陆点。如果想要避免不好的结果（并避免浪费纸飞机），更聪明的做法是不用纸飞机，而是用一架无人机，它可以感知其环境，将数据发回给操纵者，并且能够基于当前状态自主航行。我们下面要介绍的技术，可以让model.fit()的调用从纸飞机变为智能的自主无人机，可以自我反省并动态地采取行动。

### 7.2.1　训练过程中将回调函数作用于模型
训练模型时，很多事情一开始都无法预测。尤其是你不知道需要多少轮才能得到最佳验证损失。前面所有例子都采用这样一种策略：训练足够多的轮次，这时模型已经开始过拟合，根据这第一次运行来确定训练所需要的正确轮数，然后使用这个最佳轮数从头开始再启动一次新的训练。当然，这种方法很浪费。    

处理这个问题的更好方法是，当观测到验证损失不再改善时就停止训练。这可以使用  Keras回调函数来实现。回调函数（callback）是在调用fit时传入模型的一个对象（即实现特定方法的类实例），它在训练过程中的不同时间点都会被模型调用。它可以访问关于模型状态与性能的所有可用数据，还可以采取行动：中断训练、保存模型、加载一组不同的权重或改变模型的状态。    

回调函数的一些用法示例如下所示。   

模型检查点（model  checkpointing）：在训练过程中的不同时间点保存模型的当前权重。    
提前终止（early  stopping）：如果验证损失不再改善，则中断训练（当然，同时保存在训练过程中得到的最佳模型）。   
在训练过程中动态调节某些参数值：比如优化器的学习率。   
在训练过程中记录训练指标和验证指标，或将模型学到的表示可视化（这些表示也在不断更新）：你熟悉的 Keras进度条就是一个回调函数！   


#### 7.2.1.1. ModelCheckpoint与  EarlyStopping回调函数

如果监控的目标指标在设定的轮数内不再改善，可以用 EarlyStopping回调函数来中断训练。比如，这个回调函数可以在刚开始过拟合的时候就中断训练，从而避免用更少的轮次重新训练模型。这个回调函数通常与 ModelCheckpoint结合使用，后者可以在训练过程中持续不断地保存模型（你也可以选择只保存目前的最佳模型，即一轮结束后具有最佳性能的模型）。

In [8]:
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [2]:


callbacks_list=[
    keras.callbacks.EarlyStopping(#如果不再改善，就中断训练
    Monitor ='acc', #监控模型的验证精度
    patience=1, #如果精度在两轮内不再改善，则中断训练
    ),
    keras.callbacks.ModelCheckpoint(# 每轮过后保存当前权重，
        filepath='my_model.h5', # 目标模型文件的保存路径
        monitor ='val_loss', # 这两个参数的含义是如果val_loss没有改善，那么不需要覆盖模型文件，这就可以始终保存在训练
        save_best_only=True, #过程中见到的最佳模型
    )
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc']) # 你监控精度，所以它应该是模型指标的一部分

model.fit(x,y,epochs=10, # 注意回调函数要监控验证损失和验证精度，所以在调用fit时需要传入validation_data
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val,y_val))


#### 7.2.1.2. ReduceLROnPlateau回调函数

如果验证损失不再改善，你可以使用这个回调函数来降低学习率。在训练过程中如果出现了损失平台（loss plateau），那么增大或减小学习率都是跳出局部最小值的有效策略。下面这个示例使用了ReduceLROnPlateau回调函数。

In [ ]:
callbacks_list=[
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', #监控模型的验证损失
        factor=0.1, #触发时将学习率除以10
        patience=10,) #如果验证损失在10轮内都没有改善，那么就触发这个回调函数
]


model.fit(x,y, #注意因为回调函数需要监控验证损失，所以你需要在调用fit时传入valdiation_data
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val,y_val))




#### 7.2.1.3.编写你自己的回调函数
如果你需要在训练过程中采取特定行动，而这项行动又没有包含在内置回调函数中，那么可以编写你自己的回调函数。回调函数的实现方式是创建 keras.callbacks.Callback类的子类。然后你可以实现下面这些方法（从名称中即可看出这些方法的作用），它们分别在训练过程中的不同时间点被调用。  

这些方法被调用时都有一个 logs参数，这个参数是一个字典，里面包含前一个批量、前一个轮次或前一次训练的信息，即训练指标和验证指标等。此外，回调函数还可以访问下列属性。

self.model：调用回调函数的模型实例。   
self.validation_data：传入fit作为验证数据的值。   

下面是一个自定义回调函数的简单示例，它可以在每轮结束后将模型每层的激活保存到硬盘（格式为 Numpy数组），这个激活是对验证集的第一个样本计算得到的。

In [7]:
class ActivationLogger(keras.callbacks.Callback):
    def set_model(self,model):
        self.model=model # 在训练之前由父模型调用，告诉回调函数是哪个模型在调用它
        layer_outputs= [layer.output for layer in model.layers]
        self.activations_model= keras.models.Model(model.input,layer_outputs) # 模型实例，返回每层的激活
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample= self.validation_data[0][0:1] # 或许验证数据的第一个输入样本
        
        activations= self.activations_model.predict(validation_sample)
        f= open('activations_at_epoch_'+str(epoch)+'.npz','w') # 将数据保存到硬盘
        np.savez(f, activations)
        f.close()
        
        

### 7.2.2　TensorBoard简介：TensorFlow的可视化框架

想要做好研究或开发出好的模型，在实验过程中你需要丰富频繁的反馈，从而知道模型内部正在发生什么。这正是运行实验的目的：获取关于模型表现好坏的信息，越多越好。取得进展是一个反复迭代的过程（或循环）：首先你有一个想法，并将其表述为一个实验，用于验证你的想法是否正确。你运行这个实验，并处理其生成的信息。这又激发了你的下一个想法。在这个循环中实验的迭代次数越多，你的想法也就变得越来越精确、越来越强大。Keras可以帮你在最短的时间内将想法转化成实验，而高速 GPU可以帮你尽快得到实验结果。但如何处理实验结果呢？这就需要 TensorBoard发挥作用了（见图  7-9）


本节将介绍 TensorBoard，一个内置于 TensorFlow中的基于浏览器的可视化工具。注意，只有当 Keras使用  TensorFlow后端时，这一方法才能用于  Keras模型。

TensorBoard的主要用途是，在训练过程中帮助你以可视化的方法监控模型内部发生的一切。如果你监控了除模型最终损失之外的更多信息，那么可以更清楚地了解模型做了什么、没做什么，并且能够更快地取得进展。TensorBoard具有下列巧妙的功能，都在浏览器中实现。

1. 在训练过程中以可视化的方式监控指标    
2. 将模型架构可视化    
3. 将激活和梯度的直方图可视化     
4. 以三维的形式研究嵌入    

我们用一个简单的例子来演示这些功能：在  IMDB情感分析任务上训练一个一维卷积神经网络。

这个模型类似于 6.4节的模型。我们将只考虑   IMDB词表中的前  2000个单词，这样更易于将词嵌入可视化

In [9]:
max_features= 2000
max_len= 500

(x_train, y_train), (x_test, y_test)= imdb.load_data(num_words= max_features)
x_train= sequence.pad_sequences(x_train,maxlen=max_len)
x_test= sequence.pad_sequences(x_test,maxlen=max_len)

model= keras.models.Sequential()

model.add(layers.Embedding(max_features,128,
                          input_length= max_len,
                          name= 'embed'))
model.add(layers.Conv1D(32,7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32,7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])




<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/Users/yihuawang/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/yihuawang/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprec

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d (Conv1D)              (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
__________________________________________________

In [12]:
# 在开始使用 TensorBoard之前，我们需要创建一个目录，用于保存它生成的日志文件。
# 代码清单 7-8　为 TensorBoard日志文件创建一个目录

callbacks=[
    keras.callbacks.TensorBoard(
        log_dir='/Users/yihuawang/Deeplearning/my_log_dir/', #保存日志文件
        histogram_freq=1, #每一轮以后激活直方图
        embeddings_freq=1,) #每一轮以后嵌入数据
]

history= model.fit(x_train,y_train,
                   epochs=20,
                  batch_size=128,
                  validation_split=0.2,
                  callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 24s 146ms/step - loss: 7.8078 - acc: 0.4938 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 2/20
157/157 [==============================] - 22s 141ms/step - loss: 7.7602 - acc: 0.4969 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 3/20
157/157 [==============================] - 22s 138ms/step - loss: 7.7151 - acc: 0.4998 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 4/20
157/157 [==============================] - 22s 139ms/step - loss: 7.7187 - acc: 0.4996 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 5/20
157/157 [==============================] - 22s 142ms/step - loss: 7.7610 - acc: 0.4969 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 6/20
157/157 [==============================] - 22s 142ms/step - loss: 7.7260 - acc: 0.4991 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 7/20
157/157 [==============================] - 23s 145ms/step - loss: 7.7634 - acc: 0.4967 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 8/20
157/157 [==============================] - 2

In [ ]:
# 在terminal中启动服务器

$ tensorBoard --logdir=my_log_dir


### 7.2.3 小结
Keras回调函数提供了一种简单方法，可以在训练过程中监控模型并根据模型状态自动采取行动。    
使用   TensorFlow时，TensorBoard是一种在浏览器中将模型活动可视化的好方法。在Keras模型中你可以通过TensorBoard回调函数来使用这种方法。